# An introduction to `relatio` 
**Runtime $\sim$ 1h**

Original paper: ["Text Semantics Capture Political and Economic Narratives"](https://arxiv.org/abs/2108.01720)

----------------------------

This is a short demo of the package `relatio`.  It takes as input a text corpus and outputs a list of narrative statements. The pipeline is unsupervised: the user does not need to specify narratives beforehand. Narrative statements are defined as tuples of semantic roles with a (agent, verb, patient, attribute) structure. 

Here, we present the main wrapper functions to quickly obtain narrative statements from a corpus.

----------------------------

We provide datasets that have already been split into sentences and annotated by our team.

The datasets are provided in three different formats:
 1. `raw` (unprocessed)
 2. `split_sentences` (as a list of sentences)
 3. `srl` (as a list of annotated sentences by the semantic role labeler)

In this tutorial, we work with the Trump Tweet Archive corpus.

----------------------------

In [1]:
# Catch warnings for an easy ride
from relatio._logging import FileLogger
logger = FileLogger(level = 'WARNING')

In [2]:
# Browse list of available datasets
from relatio.datasets import list_datasets
print(list_datasets())

# Load an available dataset
from relatio.datasets import load_trump_data
df = load_trump_data("raw")


    List of available datasets:

    Trump Tweet Archive
    - function call: load_trump_data()
    - format: 'raw', 'split_sentences', 'srl_res'
    - allennlp version: 0.9
    - srl model: srl-model-2018.05.25.tar.gz
    


## Step 1: Split into sentences

----------------------------

For any new corpus, the first thing you will want to do is to split the corpus into sentences.

We do this on the first 100 tweets. 

The output is two lists: one with an index for the document and one with the resulting split sentences.

----------------------------


In [3]:
from relatio.preprocessing import *

p = Preprocessor(
    spacy_model = "en_core_web_md",
    remove_punctuation = True,
    remove_digits = True,
    lowercase = True,
    lemmatize = True,
    stop_words = [],
    n_process = -1,
    batch_size = 100
)

split_sentences = p.split_into_sentences(
    df.iloc[0:1000], output_path='sentences.json', progress_bar=True
)

from relatio.utils import load_sentences
split_sentences = load_sentences('sentences.json')

Splitting into sentences...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1097.51it/s]


## Step 2: Annotate semantic roles

----------------------------

Once the corpus is split into sentences. You can feed it to the semantic role labeler.

The output is a list of json objects which contain the semantic role annotations for each sentence in the corpus.

----------------------------


In [4]:
from relatio.semantic_role_labeling import *

SRL = SRL(
    path = "https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz",
    batch_size = 10,
    cuda_device = -1
)

srl_res = SRL(split_sentences[1], progress_bar=True)

Running SRL...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [01:13<00:00,  3.13it/s]


In [5]:
srl_res[0]

{'verbs': [{'verb': 'have',
   'description': 'Republicans and Democrats [V: have] both created our economic problems .',
   'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O']},
  {'verb': 'created',
   'description': '[ARG0: Republicans and Democrats] have both [V: created] [ARG1: our economic problems] .',
   'tags': ['B-ARG0',
    'I-ARG0',
    'I-ARG0',
    'O',
    'O',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']}],
 'words': ['Republicans',
  'and',
  'Democrats',
  'have',
  'both',
  'created',
  'our',
  'economic',
  'problems',
  '.']}

NB: This step is faster with a GPU. The argument cuda_device allows users to use their GPUs:

```
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())

SRL = SRL(
    path = "https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz",
    batch_size = 10,
    cuda_device = 0
)

srl_res = SRL(split_sentences[1], progress_bar=True)
```

In [6]:
# To save us some time, we download the results from the datasets module.
split_sentences = load_trump_data("split_sentences")
srl_res = load_trump_data("srl_res")

## Step 3: Pre-process semantic roles

----------------------------


----------------------------

In [7]:
roles, sentence_index = p.extract_roles(
    srl_res, 
    used_roles = ["ARG0","B-V","B-ARGM-NEG","B-ARGM-MOD","ARG1","ARG2"],
    progress_bar = True
)

for d in roles[0:5]: print(d)

Extracting semantic roles...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68616/68616 [00:01<00:00, 43840.11it/s]

{'B-V': 'have'}
{'ARG0': 'Republicans and Democrats', 'ARG1': 'our economic problems', 'B-V': 'created'}
{'ARG1': 'I', 'ARG2': 'thrilled to be back in the Great city of Charlotte , North Carolina with thousands of hardworking American Patriots who love our Country , cherish our values , respect our laws , and always put AMERICA FIRST', 'B-V': 'was'}
{'ARG1': 'I', 'ARG2': 'to be back in the Great city of Charlotte , North Carolina with thousands of hardworking American Patriots who love our Country , cherish our values , respect our laws , and always put AMERICA FIRST', 'B-V': 'thrilled'}
{'ARG1': 'I', 'ARG2': 'back in the Great city of Charlotte , North Carolina', 'B-V': 'be'}


In [8]:
postproc_roles = p.process_roles(roles, 
                                 dict_of_pos_tags_to_keep = {
                                     "ARG0": ['NOUN', 'PROPN'],
                                     "B-V": ['VERB'],
                                     "ARG1": ['NOUN', 'PROPN'],
                                     "ARG2": ['NOUN', 'PROPN']
                                 }, 
                                 progress_bar = True,
                                 output_path = 'postproc_roles.json')

from relatio.utils import load_roles
postproc_roles = load_roles('postproc_roles.json')

for d in postproc_roles[0:5]: print(d)

Cleaning roles ARG0...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49281/49281 [00:15<00:00, 3158.48it/s]


Cleaning roles B-V...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 135562/135562 [00:41<00:00, 3286.97it/s]


Cleaning roles B-ARGM-MOD...


...

Cleaning roles ARG1...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88730/88730 [00:31<00:00, 2776.80it/s]


Cleaning roles ARG2...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32132/32132 [00:12<00:00, 2563.23it/s]


{'B-V': 'have'}
{'ARG0': 'republicans democrats', 'B-V': 'create', 'ARG1': 'problem'}
{'B-V': '', 'ARG1': '', 'ARG2': 'city charlotte north carolina thousand patriots country value law america'}
{'B-V': 'thrill', 'ARG1': '', 'ARG2': 'city charlotte north carolina thousand patriots country value law america'}
{'B-V': 'be', 'ARG1': '', 'ARG2': 'city charlotte north carolina'}


In [9]:
known_entities = p.mine_entities(
    split_sentences[1], 
    clean_entities = True, 
    progress_bar = True,
    output_path = 'entities.pkl'
)

from relatio.utils import load_entities
known_entities = load_entities('entities.pkl')

for n in known_entities.most_common(10): print(n)

Mining named entities...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68616/68616 [00:34<00:00, 1969.47it/s]

('democrats', 1149)
('obama', 1023)
('china', 932)
('u.s.', 836)
('trump', 798)
('america', 768)
('american', 589)
('barackobama', 512)
('republicans', 502)
('foxnews', 502)


In [10]:
top_known_entities = [e[0] for e in list(known_entities.most_common(100))]

## Step 4: Build the narrative model

----------------------------

We are now ready to build a narrative model.

----------------------------

In [11]:
from relatio._embeddings import Embeddings
nlp_model = Embeddings("TensorFlow_USE","https://tfhub.dev/google/universal-sentence-encoder/4")

2022-03-01 18:15:13.772916: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-01 18:15:13.805599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-01 18:15:13.805887: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-01 18:15:13.806661: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [28]:
from relatio.narrative_models import *
from relatio.utils import prettify
from collections import Counter

m = NarrativeModel(model_type = 'deterministic',
                   roles_considered = ['ARG0', 'B-V', 'B-ARGM-NEG', 'B-ARGM-MOD', 'ARG1', 'ARG2'],
                   roles_with_entities = ['ARG0','ARG1','ARG2'],
                   list_of_known_entities = top_known_entities,
                   assignment_to_known_entities = 'character_matching',
                   roles_with_embeddings = [],
                   embeddings_model = nlp_model,
                   threshold = 1)    

m.train(postproc_roles)

No training required: the model is deterministic.


In [29]:
import cProfile
cProfile.run("narratives = m.predict(postproc_roles, progress_bar = True, prettify = False)")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150213/150213 [00:16<00:00, 9241.28it/s]


         73992935 function calls (73190256 primitive calls) in 17.338 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.009    0.009   17.338   17.338 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _monitor.py:93(report)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:106(remove)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:16(__init__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:20(__enter__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:26(__exit__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:52(_commit_removals)
        3    0.000    0.000    0.000    0.000 _weakrefset.py:58(__iter__)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:81(add)
 802680/1    0.518    0.000    1.070    1.070 copy.py:128(deepcopy)
   652466    0.047    0.000    0.047    0.000 copy.py:182(_deepcopy_atomic)
        1    0.045

In [30]:
pretty_narratives = []
for n in narratives: 
    if n.get('ARG0') not in ["", None]:
        if n.get('B-V') not in ["", None]:
            if n.get('ARG1') not in ["", None]:
                pretty_narratives.append(prettify(n))
                
pretty_narratives = Counter(pretty_narratives)
for t in pretty_narratives.most_common(10): print(t)

('house pass bill', 4)
('china take u.s.', 4)
('pelosi demand republicans|house', 3)
('dem want senate', 3)
('dem run senate', 3)
('north carolina make republican', 3)
('foxnews discuss barackobama', 3)
('canada look china', 3)
('foxnews discuss mittromney', 3)
('iran take iraq', 3)


In [18]:
m = NarrativeModel(model_type = 'dynamic',
                   roles_considered = ['ARG0', 'B-V', 'B-ARGM-NEG', 'B-ARGM-MOD', 'ARG1', 'ARG2'],
                   roles_with_entities = ['ARG0','ARG1','ARG2'],
                   list_of_known_entities = top_known_entities,
                   assignment_to_known_entities = 'embeddings',
                   roles_with_embeddings = [['ARG0','ARG1','ARG2']],
                   embeddings_model = None,
                   threshold = 0.1)    

m.train(postproc_roles, progress_bar = True)

Computing vectors for known entities...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150213/150213 [17:03<00:00, 146.81it/s]


In [19]:
cProfile.run("narratives = m.predict(postproc_roles, progress_bar = True, prettify = False)")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150213/150213 [29:39<00:00, 84.42it/s]


         388039084 function calls (359710073 primitive calls) in 1780.073 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   109589    0.179    0.000    1.808    0.000 <__array_function__ internals>:2(amax)
   109589    0.202    0.000    3.881    0.000 <__array_function__ internals>:2(amin)
   279732    0.263    0.000    2.083    0.000 <__array_function__ internals>:2(atleast_1d)
   219178    0.187    0.000    1.342    0.000 <__array_function__ internals>:2(atleast_2d)
  1266033    1.102    0.000    7.128    0.000 <__array_function__ internals>:2(concatenate)
   170143    0.204    0.000    0.662    0.000 <__array_function__ internals>:2(count_nonzero)
   109589    0.097    0.000    0.526    0.000 <__array_function__ internals>:2(dot)
   279732    0.275    0.000    5.453    0.000 <__array_function__ internals>:2(hstack)
   109589    0.143    0.000    2.046    0.000 <__array_function__ internals>:2(norm)
   242216    0.232   

In [23]:
pretty_narratives = []
for n in narratives: 
    if n.get('ARG0') not in ["", None]:
        if n.get('B-V') not in ["", None]:
            if n.get('ARG1') not in ["", None]:
                pretty_narratives.append(prettify(n))
                
pretty_narratives = Counter(pretty_narratives)
for t in pretty_narratives.most_common(10): print(t)

('jim have complete total endorsement', 9)
('scott have complete total endorsement', 8)
('mike have complete total endorsement', 8)
('establishment interest kill country', 8)
('china want deal', 8)
('god bless u.s.a.', 7)
('gold make rule', 7)
('china make deal', 7)
('people love country', 6)
('tom have complete total endorsement', 6)


## Step 5: Model validation and basic analysis

----------------------------


----------------------------

In [21]:
for i,k in enumerate(sentence_index):
    
    n = narratives[i]
    r = roles[i]
    
    if n.get('ARG0') not in ["", None]:
        if n.get('B-V') not in ["", None]:
            if n.get('ARG1') not in ["", None]:
                print('Original statement:')
                print(prettify(r))
                print('\n')
                print('Underlying narrative:')
                print(prettify(n))
                print("\n")
                

Original statement:
Republicans and Democrats created our economic problems


Underlying narrative:
republicans democrats create problem


Original statement:
hardworking American Patriots who love our Country


Underlying narrative:
patriots love country


Original statement:
hardworking American Patriots who cherish our values


Underlying narrative:
patriots cherish value


Original statement:
hardworking American Patriots who put AMERICA FIRST


Underlying narrative:
patriots put america


Original statement:
Almost all recent elections using this system


Underlying narrative:
election use system


Original statement:
Sudan agreed to a peace and normalization agreement with Israel


Underlying narrative:
sudan agree peace normalization agreement israel


Original statement:
AdamLaxalt finding things that , when released , will be absolutely shocking


Underlying narrative:
adamlaxalt find thing


Original statement:
SenTomCotton Republicans have pluses & amp ; minuses


Underlying

Underlying narrative:
biden not get vote


Original statement:
by Obama - Biden Auto Bailout Forgotten Delphi Workers


Underlying narrative:
obama biden auto bailout forget delphi worker


Original statement:
Delphi Workers Refuse to Forget What Was Taken from Them ( Part One ) via


Underlying narrative:
delphi worker refuse part


Original statement:
Delphi Workers Forget What Was Taken from Them ( Part One )


Underlying narrative:
delphi worker forget part


Original statement:
Lesley Stahl of 60 Minutes not wearing a mask


Underlying narrative:
lesley stahl minutes not wear mask


Original statement:
Biden and Harris Pack the Court


Underlying narrative:
biden harris pack court


Original statement:
Justice Barrett Apply Will the Law


Underlying narrative:
justice barrett apply will law


Original statement:
Democrats suffered crushing down - ballot loss across America


Underlying narrative:
democrats suffer ballot loss america


Original statement:
Judge Brann who not allow 


Original statement:
the PA Secretary of State announced that there are Millions of ballots left to be counted


Underlying narrative:
pa secretary state announce million ballot


Original statement:
the Commonwealth which not allow wo legal observers


Underlying narrative:
commonwealth not allow wo observer


Original statement:
the person who said Libyan solution


Underlying narrative:
person say solution


Original statement:
Our lawyers asked for meaningful access


Underlying narrative:
lawyer ask access


Original statement:
Congress not giving a Stimulus Bill our people


Underlying narrative:
congress not give stimulus bill people


Original statement:
Trump Won Would Nevada


Underlying narrative:
trump won would nevada


Original statement:
Congress cut should the pork


Underlying narrative:
congress cut should pork


Original statement:
a Democrat Presidential Candidate had an Election Rigged & amp ; Stolen


Underlying narrative:
democrat presidential candidate have elec

second amendment vets military new york law enforcement elise have complete total endorsement


Original statement:
Congressman BryanSteil doing a great job for the people of Wisconsin


Underlying narrative:
congressman bryansteil do job people wisconsin


Original statement:
Bryan has my Complete and Total Endorsement


Underlying narrative:
bryan have complete total endorsement


Original statement:
Joe Biden cut your Social Security and Medicarea fact which Joe falsely denied last night


Underlying narrative:
joe biden cut social security medicarea fact joe night


Original statement:
Joe denied your Social Security and Medicarea fact which


Underlying narrative:
joe deny social security medicarea fact


Original statement:
by Bernie Sanders created This video ( Joe Bidens own words )


Underlying narrative:
bernie sanders create video joe bidens word


Original statement:
Few people fought for Iowa


Underlying narrative:
people fight iowa


Original statement:
Judiciary Committ


Underlying narrative:
gd build arleigh burke destroyer


Original statement:
Romney hurt some very good Republican Senators


Underlying narrative:
romney hurt senator


Original statement:
The Department of Education not closing hundreds of schools in rural areas of our Country


Underlying narrative:
department education not close hundred school area country


Original statement:
Gallup gave the highest rating ever for the way we are handling the CoronaVirus situation us


Underlying narrative:
gallup give rating way coronavirus situation


Original statement:
The States get the money to the people who need it


Underlying narrative:
states get money people


Original statement:
The Wacky Do Nothing Attorney General of Michigan , Dana Nessel , threatening Ford Motor Company for the fact that I inspected a Ventilator plant without a mask


Underlying narrative:
wacky attorney general michigan dana nessel threaten ford motor company fact ventilator plant mask


Original statement:
man

...


Original statement:
Democrats raise your taxes


Underlying narrative:
democrats raise taxis


Original statement:
the media claim would that I had a Messiah complex


Underlying narrative:
medium claim would messiah complex


Original statement:
the Fake News Reporters that not have a clue as to what the law is relative to Presidential powers , China , etc .


Underlying narrative:
fake news reporter not have clue law power china


Original statement:
by Lous friend , DavidAsmanfox hosted LouDobbs


Underlying narrative:
lous friend davidasmanfox host loudobbs


Original statement:
the United States help can with the Amazon Rainforest fires


Underlying narrative:
united states help can amazon rainforest fire


Original statement:
China not put should new Tariffs on 75 BILLION DOLLARS of United States product ( politically motivated ! )


Underlying narrative:
china not put should tariff dollar united states product


Original statement:
past Administrations allowed China to get so f

the others who did so many other terrible things


Underlying narrative:
other do thing


Original statement:
the Democrats in the House calling Comey , Brennan , Clapper , Page and her FBI lover ( whose invaluable phone records were illegally deleted ) , Crooked Hillary , Podesta , Ohr ( and Nellie ) , the GPS Fusion characters , Christopher Steele , the DNC ( & amp , their missing server )


Underlying narrative:
democrats house call comey brennan clapper page fbi lover phone record crooked hillary podesta ohr nellie gps fusion character christopher steele dnc amp server


Original statement:
The Republican Senate passed bipartisan humanitarian assistance for our Southern Border , 84 - 8


Underlying narrative:
senate pass assistance southern border


Original statement:
the catastrophic loopholes that driving the Crisis


Underlying narrative:
loophole drive crisis


Original statement:
Democrats say hearings could change impeachment debate


Underlying narrative:
democrats say hear

state department say security incidents employee handling hillary clintons emails


Original statement:
U.S. escalating attacks


Underlying narrative:
u.s. escalate attack


Original statement:
ICE begin will the process of removing the millions of illegal aliens who have illicitly found their way into the United States


Underlying narrative:
ice begin will process million alien way united states


Original statement:
ICE removing the millions of illegal aliens who have illicitly found their way into the United States


Underlying narrative:
ice remove million alien way united states


Original statement:
illegal aliens who found their way into the United States


Underlying narrative:
alien find way united states


Original statement:
Mexico using their strong immigration laws


Underlying narrative:
mexico use immigration law


Original statement:
Mexico doing a very good job of stopping people


Underlying narrative:
mexico do job people


Original statement:
Our polls show us lea

Original statement:
Saudi Arabia and others in OPEC make will the Oil Flow difference


Underlying narrative:
saudi arabia other opec make will oil flow difference


Original statement:
by JohnKerry and people who helped him lead the U.S. into the very bad Iran Nuclear Deal given VERY BAD advice Iran


Underlying narrative:
johnkerry people u.s. iran nuclear deal give advice iran


Original statement:
JohnKerry and people who helped him lead the U.S. into the very bad Iran Nuclear Deal


Underlying narrative:
johnkerry people help u.s. iran nuclear deal


Original statement:
by the Democrats committed a crime that


Underlying narrative:
democrats commit crime


Original statement:
the other side that created the diversionary & amp , criminal event


Underlying narrative:
side create diversionary amp event


Original statement:
the other side that spied on my campaign


Underlying narrative:
side spy campaign


Original statement:
Jobless claims in the United States reached their lowes


Original statement:
some Democrats keep the Witch Hunt alive


Underlying narrative:
democrats keep witch hunt


Original statement:
by the Fed taken the unnecessary and destructive actions


Underlying narrative:
fed take action


Original statement:
The First Step Act proves that our Country can achieve amazing breakthroughs when we put politics aside , and put the interests of ALL Americans FIRST


Underlying narrative:
first step act prove country breakthrough politic interest americans


Original statement:
our Country achieve can amazing breakthroughs


Underlying narrative:
country achieve can breakthrough


Original statement:
Congress eliminate must the loopholes at the Border


Underlying narrative:
congress eliminate must loophole border


Original statement:
some very talented people developing the wonderful HealthCare package that


Underlying narrative:
people develop healthcare package


Original statement:
every member of our society reach their full potential


Underl

## Step 6: Visualization // Plotting narrative graphs
----------------------------

A collection of narrative statements has an intuitive network structure, in which the edges are verbs and the nodes are entities.

Here, we plot Trump's narrative statements on Twitter.

----------------------------